# L3 — Chains and Runnables (LangChain v1)

# Setup

This notebook uses **OpenAI (Python SDK v2) + LangChain v1**.

## Prereqs
1. Set your API key in the environment:

```bash
export OPENAI_API_KEY="..."
```

2. Restart the kernel after setting env vars.


In [ ]:
import os

# Make sure your key is set
assert os.getenv("OPENAI_API_KEY"), "Set OPENAI_API_KEY in your environment before running."

MODEL = "gpt-5-mini"


We'll build a small pipeline:
1) generate an outline
2) expand one section
3) summarize

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model=MODEL)
to_text = StrOutputParser()

outline_prompt = ChatPromptTemplate.from_messages([
    ("system", "Create a compact outline with 4 bullet points."),
    ("user", "{topic}")
])

expand_prompt = ChatPromptTemplate.from_messages([
    ("system", "Expand bullet point #2 into a short paragraph."),
    ("user", "Outline:\n{outline}")
])

summ_prompt = ChatPromptTemplate.from_messages([
    ("system", "Summarize in 2 sentences."),
    ("user", "{text}")
])

outline_chain = outline_prompt | llm | to_text
expand_chain = expand_prompt | llm | to_text
summ_chain = summ_prompt | llm | to_text

chain = (
    {"topic": RunnablePassthrough()}
    | RunnableLambda(lambda x: x["topic"])
    | outline_chain
    | (lambda outline: {"outline": outline, "text": outline})
    | RunnableLambda(lambda d: d)
)

topic = "How to write reliable prompts for LLMs"
outline = outline_chain.invoke({"topic": topic})
expanded = expand_chain.invoke({"outline": outline})
summary = summ_chain.invoke({"text": expanded})

print("OUTLINE:\n", outline)
print("\nEXPANDED:\n", expanded)
print("\nSUMMARY:\n", summary)
